In [1]:
import os
import gc
import cv2
import time
import json
import paddle
import numpy as np
import pandas as pd
import paddle.nn as nn
import matplotlib.pyplot as plt
import paddle.nn.functional as F
from paddle.io import DataLoader, TensorDataset
from datetime import datetime
import sys
import math

time1 = datetime.now()
save_dir = f'./Save_Directory/10_22SHARP_iTransformer_Test/{time1}/'
os.makedirs(save_dir, exist_ok=True)

In [2]:
class Embedding_inverted(nn.Layer):
    # intput_dim: [B, L, in_channels] B为batch_size，L为timestep，in_channels为特征数
    def __init__(self, in_channels=10, embed_dim=128, dropout=0.1):
        super(Embedding_inverted, self).__init__()
        self.embedding = nn.Linear(in_channels, embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x = x.transpose([0, 2, 1])   #[B, Variate, Time]
        x = self.embedding(x)
        x = self.dropout(x)
        return x
    # 输出的形状为 [B, L, embed_dim]

class FullAttention(nn.Layer):
    def __init__(self, scale=None, attention_dropout=0.1):
        super(FullAttention, self).__init__()
        self.scale = scale
        self.dropout = nn.Dropout(attention_dropout)

    # 输入：Q,K,V的形状为 [B, L, H, E]  [batch_size,timestep,Number of Heads,Embedding Size Per Head]
    def forward(self, Q, K, V):
        B, L, H, E = Q.shape
        _, S, _, D = V.shape
        scale = self.scale if self.scale is not None else 1. / math.sqrt(E)

        attn = paddle.einsum("blhe,bshe->bhls", Q, K)
        attn = F.softmax(attn * scale, axis=-1)
        attn = self.dropout(attn)

        v = paddle.einsum("bhls,bshd->blhd", attn, V)
        return v.contiguous()
        # 输出的形状为 [B, L, H, D]  D是值向量的维度

class AttentionLayer(nn.Layer):
    def __init__(self, embed_dim=128, n_heads=3, attention_dropout=0.1, d_keys=None, d_values=None):
        super(AttentionLayer, self).__init__()
        d_keys = d_keys or (embed_dim // n_heads)
        d_values = d_values or (embed_dim // n_heads)

        self.inner_attention = FullAttention(scale=None, attention_dropout=attention_dropout)
        self.q_projection = nn.Linear(embed_dim, d_keys * n_heads)
        self.k_projection = nn.Linear(embed_dim, d_keys * n_heads)
        self.v_projection = nn.Linear(embed_dim, d_values * n_heads)
        self.out_projection = nn.Linear(d_values * n_heads, embed_dim)
        self.n_heads = n_heads

    def forward(self, Q, K, V):
        # 输入：查询、键和值，形状均为 [B, L, embed_dim]
        B, L, _ = Q.shape
        _, S, _ = K.shape
        H = self.n_heads

        q = self.q_projection(Q).reshape((B, L, H, -1))
        k = self.k_projection(K).reshape((B, S, H, -1))
        v = self.v_projection(V).reshape((B, S, H, -1))

        out = self.inner_attention(q, k, v)
        out = out.reshape((B, L, -1))
        out = self.out_projection(out)
        return out
        # 输出的形状为 [B, L, embed_dim]    


class EncoderLayer(nn.Layer):
    def __init__(self, embed_dim=128, mlp_ratio=None, dropout=0.1, n_heads=3):
        super(EncoderLayer, self).__init__()
        d_ff = int(mlp_ratio * embed_dim)
        self.attention = AttentionLayer(embed_dim, n_heads, dropout)
        self.conv1 = nn.Conv1D(in_channels=embed_dim, out_channels=d_ff, kernel_size=1)
        self.conv2 = nn.Conv1D(in_channels=d_ff, out_channels=embed_dim, kernel_size=1)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)
        self.activation = F.gelu  

    def forward(self, x):
        # 输入 x 的形状为 [B, L, embed_dim]
        new_x = self.attention(x, x, x) # 计算多头注意力
        x = x + self.dropout(new_x)  # 残差连接
        x = self.norm1(x)

        y = x
        y = self.conv1(y.transpose([0, 2, 1]))
        y = self.activation(y)
        y = self.dropout(y)
        y = self.conv2(y).transpose([0, 2, 1])
        y = self.dropout(y)
        y = self.norm2(x + y)
        return y
        # 输出的形状为 [B, L, embed_dim]

class Encoder(nn.Layer):
    def __init__(self, embed_dim=128, mlp_ratio=4.0, dropout=0.1, n_heads=3, depth=12):
        super(Encoder, self).__init__()
        attn_layers = [
            EncoderLayer(embed_dim, mlp_ratio=mlp_ratio, dropout=dropout, n_heads=n_heads) for _ in range(depth)
        ]
        self.attn_layers = nn.LayerList(attn_layers)
        self.norm = nn.LayerNorm(embed_dim)
    
    def forward(self, x):
        for attn_layer in self.attn_layers:
            x = attn_layer(x)
        x = self.norm(x)
        return x


class iTransformer(nn.Layer):
    def __init__(self, 
                 in_channels=1,
                 num_features=5,
                 embed_dim=128,
                 n_heads=3,
                 depth=12,
                 mlp_ratio=4.0,
                 dropout=0.1,
                 num_class=2):
        super(iTransformer, self).__init__()
        
        self.embed_dim = embed_dim
        self.enc_embedding = Embedding_inverted(in_channels, embed_dim, dropout)
        # self.enc_embedding2 = Embedding_inverted(num_features, embed_dim, dropout)
        self.encoder = Encoder(embed_dim, mlp_ratio=mlp_ratio, dropout=dropout, n_heads=n_heads, depth=depth)
        self.act = F.gelu
        self.dropout = nn.Dropout(dropout)
        self.projection = nn.Linear(embed_dim, num_class)
        

    def classification(self, x):
        enc_out = self.enc_embedding(x)
        # print(enc_out.shape)
        # enc_out2 = self.enc_embedding2(x.transpose([0, 2, 1]))
        # enc_out = paddle.concat([enc_out1, enc_out2], axis=1)
        enc_out= self.encoder(enc_out)
        # print(enc_out.shape)
        enc_out = enc_out.transpose([0,2,1])
        enc_out = F.adaptive_avg_pool1d(enc_out, 1).reshape((-1, self.embed_dim))

        output = self.act(enc_out)
        output = self.dropout(output)
        output = output.reshape((output.shape[0], -1))
        # print(output.shape)
        output = self.projection(output)
        return output

    def forward(self, x_enc):
        dec_out = self.classification(x_enc)
        
        return dec_out
        # 输出[B, num_class]

In [3]:
class MetricsCalculator(object):  
    def __init__(self):  
        self.TP = 0  
        self.FP = 0  
        self.FN = 0  
        self.TN = 0
        self.y_trues = []
        self.y_true_onehot = []
        self.y_preds_proba = []
  
    def update(self, y_true, y_pred, y_true_onehot, y_pred_proba):  
        y_true = np.array(y_true, dtype='float64').reshape(-1, 1)  
        y_pred = np.array(y_pred, dtype='float64').reshape(-1, 1)
        y_true_onehot = np.array(y_true_onehot, dtype='float64').reshape(-1, 2)
        y_pred_proba = np.array(y_pred_proba, dtype='float64').reshape(-1, 2)

        self.y_trues.extend(y_true)
        self.y_true_onehot.extend(y_true_onehot)
        self.y_preds_proba.extend(y_pred_proba)  # Update the correct variable here
  
        # 假设 y_true 中 1 表示正类，0 表示负类  
        self.TP += np.sum((y_true == 1) & (y_pred == 1)) 
        self.FN += np.sum((y_true == 1) & (y_pred == 0))
        self.FP += np.sum((y_true == 0) & (y_pred == 1))
        self.TN += np.sum((y_true == 0) & (y_pred == 0))

    def calculate_brier_score(self):
        y_true = np.array(self.y_true_onehot)
        y_pred_proba = np.array(self.y_preds_proba)  # Use the correct variable here

        # 计算 BS
        BS = np.mean((y_true - y_pred_proba) ** 2, axis=0)
        BS = BS[1]

        # 计算 BSS
        y_mean = np.mean(y_true, axis=0)
        reference_bs = np.mean((y_true - y_mean) ** 2, axis=0)
        reference_bs = np.mean((y_true[:, 1] - y_mean[1]) ** 2, axis=0)
        BSS = 1 - BS / reference_bs if reference_bs != 0 else 0

        return BS, BSS

    def calculate_metrics(self):  
        total = self.TP + self.FN + self.FP + self.TN  
  
        Accuracy = (self.TP + self.TN) / total if total > 0 else 0  
        Precision = self.TP / (self.TP + self.FP) if (self.TP + self.FP) > 0 else 0  
        Recall = self.TP / (self.TP + self.FN) if (self.TP + self.FN) > 0 else 0  
        FAR = self.FP / (self.FP + self.TP) if (self.FP + self.TP) > 0 else 0
        FPR = self.FP/(self.FP + self.TN) if (self.FP + self.TN) > 0 else 0
        TSS = Recall - (self.FP / (self.FP + self.TN)) if (self.FP + self.TN) > 0 else 0
        HSS = (2 * (self.TP * self.TN - self.FP * self.FN)) / ((self.TP + self.FN) * (self.FN + self.TN) + (self.TP + self.FP) * (self.FP + self.TN) + 1e-5)  
        
        BS, BSS = self.calculate_brier_score()
  
        metrics = {  
            'TP': self.TP,  
            'FP': self.FP,  
            'FN': self.FN,  
            'TN': self.TN,  
            'Accuracy': Accuracy,  
            'Precision': Precision,  
            'Recall': Recall,  
            'FAR': FAR,
            'FPR': FPR,  
            'TSS': TSS,  
            'HSS': HSS,
            'Brier Score (BS)': BS,
            'Brier Skill Score (BSS)': BSS
        }  
        return metrics 

In [4]:
def data_preprocess(data_path):
    data = pd.read_csv(data_path,low_memory=False)
    physics_param = data.iloc[:, 0:31].values
    labels = data.iloc[:, 34]
    label_mapping = {'N': 0, 'C': 0, 'M': 1, 'X': 1}
    labels = np.vectorize(label_mapping.get)(labels).astype('int32').reshape(-1, 1)
    return physics_param.reshape(-1, 40, 31).astype('float32'), labels[39::40, :]


In [5]:
paddle.seed(40)
np.random.seed(42)
paddle.device.cuda.empty_cache()
weight = paddle.to_tensor([0.569, 4.120], dtype='float32')

W1101 18:02:50.360286 307656 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.0, Runtime API Version: 11.8
W1101 18:02:50.361851 307656 gpu_resources.cc:164] device: 0, cuDNN Version: 8.9.


In [6]:
total_samples = 445
batch_size = 63
batch_val = total_samples//batch_size  # 总样本数445 除以 batch_size
in_channels = 6
num_features = 6
feature_start = 25
feature_end = 29
mode = 1

In [7]:
# 开始训练
for dataset_id in range(10):
    paddle.seed(40)
    np.random.seed(42)
    paddle.device.cuda.empty_cache()
    print(f'开始第{dataset_id}个数据集')
    best_TSS = -1
    best_BSS = -100
    Train_Loss = []
    Val_Loss = []
    # Data preprocessing
    
    train_data, train_label = data_preprocess(f'/home/aistudio/Dataset/10_22_SHARP/{dataset_id}Train_all_class.csv')
    val_data, val_label = data_preprocess(f'/home/aistudio/Dataset/10_22_SHARP/{dataset_id}Val_all_class.csv')
    # train_data, train_label = data_preprocess(f'/home/aistudio/Datasets_New/10_22SHARP_10Features/8Train.csv')
    # val_data, val_label = data_preprocess(f'/home/aistudio/Datasets_New/10_22SHARP_10Features/8Val.csv')


    paddle.device.cuda.empty_cache()
    gc.collect()

    train_dataset = TensorDataset([train_data, train_label])
    val_dataset = TensorDataset([val_data, val_label])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=128, shuffle=True)

    model = iTransformer(in_channels=in_channels,
                 num_features=num_features,
                 embed_dim=192,
                 n_heads=4, #4或8
                 depth=5,
                 mlp_ratio=4.0,
                 dropout=0.1,
                 num_class=2)

    scheduler = paddle.optimizer.lr.LinearWarmup(
        learning_rate=1e-3, 
        warmup_steps=10, 
        start_lr=1e-4, 
        end_lr=1e-3, 
        verbose=False)
    opt = paddle.optimizer.AdamW(
        learning_rate=scheduler, 
        weight_decay=0.1, 
        parameters=model.parameters())

    for i in range(50):
        batch_id = 0
        total_loss = 0
        model.train()

        for img, label in train_loader:
            # print(img.shape)
            batch_id += 1
            label = label.astype('int32')
            if mode == 0:
                pred = model(img[:, :, feature_start:feature_end])  # 修改
            else:
                pred = model(paddle.concat([img[:, :, feature_start:feature_end], img[:, :, 29:31]], axis=-1))
            # pred = model(paddle.concat([img[:, :, :, 1:2], img[:, :, :, 4:5],img[:, :, :, 22:27], img[:, :, :, 28:30]], axis=-1))
            loss = F.cross_entropy(pred, label, weight=weight)
            total_loss += loss.item()

            opt.clear_gradients()
            loss.backward()
            opt.step()

            if batch_id % batch_val == 0:
                print(f'Epoch: {i+1}')
                print(f'Train_Loss: {total_loss / batch_val}')
                Train_Loss.append(total_loss / batch_val)
                total_loss = 0

                TSS = []
                val_batch_id = 0
                calculator = MetricsCalculator()
                model.eval()

                with paddle.no_grad():
                    for img, label in val_loader:
                        val_batch_id += 1
                        if mode == 0:
                            pred = model(img[:, :, feature_start:feature_end])  # 修改
                        else:
                            pred = model(paddle.concat([img[:, :, feature_start:feature_end], img[:, :, 29:31]], axis=-1))
                        # pred = model(paddle.concat([img[:, :, :, 1:2], img[:, :, :, 4:5],img[:, :, :, 22:27], img[:, :, :, 28:30]], axis=-1))
                        label = label.astype('int32')
                        loss = F.cross_entropy(pred, label, weight=weight)
                        total_loss += loss.item()
                        pred_label = paddle.argmax(pred, axis=-1)
                        y_pred_proba = F.softmax(pred, axis=-1)
                        true_onehot = F.one_hot(label, 2)
                        calculator.update(label, pred_label, true_onehot, y_pred_proba)

                val_loss_avg = total_loss / val_batch_id
                print(f'Val_Loss: {val_loss_avg}')
                Val_Loss.append(val_loss_avg)
                total_loss = 0

                metric = calculator.calculate_metrics()
                if best_TSS < metric['TSS']:
                    best_TSS = metric['TSS']
                    paddle.save(model.state_dict(), f'{save_dir}model/iTransformer_sift_TSS/iTransformer_{dataset_id}.pdparam')

                if best_BSS < metric['Brier Skill Score (BSS)']:
                    best_BSS = metric['Brier Skill Score (BSS)']
                    paddle.save(model.state_dict(), f'{save_dir}model/iTransformer_sift_BSS/iTransformer_{dataset_id}.pdparam')
                print(metric)
                print('Mean TSS: {}   Best_TSS:{}'.format(metric['TSS'], best_TSS))              
                print('Mean BSS: {}   Best BSS:{}\n'.format(metric['Brier Skill Score (BSS)'], best_BSS))                

        # Clear CUDA cache and collect garbage
        paddle.device.cuda.empty_cache()
        gc.collect()

        scheduler.step()

    loss_folder = f'{save_dir}Loss/ViT_sift_BSS/'
    if not os.path.exists(loss_folder):
        os.makedirs(loss_folder)
    np.save(loss_folder + f'Train_Loss_{dataset_id}.npy', np.array(Train_Loss))
    np.save(loss_folder + f'Val_Loss_{dataset_id}.npy', np.array(Val_Loss))    

    # Clear CUDA cache and collect garbage after each dataset
    paddle.device.cuda.empty_cache()
    gc.collect()

    del train_data, train_label, val_data, val_label
    del train_dataset, val_dataset, train_loader, val_loader
    del model, scheduler, opt, calculator
    gc.collect()

Mean TSS: 0.42592592592592593   Best_TSS:0.5925925925925927
Mean BSS: -0.45657964818750063   Best BSS:0.09011785959918062

Epoch: 9
Train_Loss: 0.50090149470738
Val_Loss: 0.48316559195518494
{'TP': 9, 'FP': 18, 'FN': 3, 'TN': 90, 'Accuracy': 0.825, 'Precision': 0.3333333333333333, 'Recall': 0.75, 'FAR': 0.6666666666666666, 'FPR': 0.16666666666666666, 'TSS': 0.5833333333333334, 'HSS': 0.37499999906994047, 'Brier Score (BS)': 0.11210778543619447, 'Brier Skill Score (BSS)': -0.24564206040216074}
Mean TSS: 0.5833333333333334   Best_TSS:0.5925925925925927
Mean BSS: -0.24564206040216074   Best BSS:0.09011785959918062

Epoch: 10
Train_Loss: 0.5236348339489528
Val_Loss: 0.4367293417453766
{'TP': 8, 'FP': 17, 'FN': 4, 'TN': 91, 'Accuracy': 0.825, 'Precision': 0.32, 'Recall': 0.6666666666666666, 'FAR': 0.68, 'FPR': 0.1574074074074074, 'TSS': 0.5092592592592592, 'HSS': 0.3437499991048177, 'Brier Score (BS)': 0.13832516828507346, 'Brier Skill Score (BSS)': -0.536946314278594}
Mean TSS: 0.509259259

In [8]:
for dataset_id in range(10):
    test_data, test_label = data_preprocess(f'/home/aistudio/Dataset/10_22_SHARP/{dataset_id}Test_all_class.csv')
    # test_data, test_label = SDO_ASO_test_data_preprocess(f'/home/aistudio/dataset/SDO_physics_features_Norm.csv')
    test_dataset = TensorDataset([test_data, test_label])
    test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)
    model = iTransformer(in_channels=in_channels,
                 num_features=num_features,
                 embed_dim=192,
                 n_heads=4, #4或8
                 depth=5,
                 mlp_ratio=4.0,
                 dropout=0.1,
                 num_class=2)
    calculator = MetricsCalculator()
    param = paddle.load(f'{save_dir}model/iTransformer_sift_TSS/iTransformer_{dataset_id}.pdparam')
    model.load_dict(param)
    model.eval()

    TSS = []
    # y_true = []
    # y_perd = []
    with paddle.no_grad():
        for img, label in test_loader:
            if mode == 0:
                pred = model(img[:, :, feature_start:feature_end])  # 修改
            else:
                pred = model(paddle.concat([img[:, :, feature_start:feature_end], img[:, :, 29:31]], axis=-1))
            # pred = model(paddle.concat([img[:, :, :, 1:2], img[:, :, :, 4:5],img[:, :, :, 22:27], img[:, :, :, 28:30]], axis=-1))
            label = label.astype('int32')
            pred_label = paddle.argmax(pred, axis=-1)
            pred_proba = F.softmax(pred, axis=-1)
            true_onehot = F.one_hot(label, 2)
            # y_true.append(pred_onehot)
            # pred_proba = F.softmax(pred, axis=-1)
            # y_perd.append(pred_proba)            
            calculator.update(label, pred_label, true_onehot, pred_proba)
    TSS_csv_path = f'{save_dir}TSS_csv/'
    os.makedirs(TSS_csv_path, exist_ok=True)
    pd.DataFrame(calculator.y_true_onehot).to_csv(f'{TSS_csv_path}y_true_{str(dataset_id)}_timestep_40.csv',index=False,header=False)
    pd.DataFrame(calculator.y_preds_proba).to_csv(f'{TSS_csv_path}y_pred_{str(dataset_id)}_timestep_40.csv',index=False,header=False)
    metric = calculator.calculate_metrics()
    print(metric)

    metric_folder = f'{save_dir}Metrics_timesteps_40/iTransformer_sift_TSS/'
    if not os.path.exists(metric_folder):
        os.makedirs(metric_folder)
    data_serializable = {k: int(v) if isinstance(v, np.integer) else v for k, v in metric.items()} 
    with open(metric_folder + f'dataset_{dataset_id}.json', 'w', encoding='utf-8') as f:  
        json.dump(data_serializable, f, ensure_ascii=True, indent=4)    

{'TP': 17, 'FP': 12, 'FN': 3, 'TN': 79, 'Accuracy': 0.8648648648648649, 'Precision': 0.5862068965517241, 'Recall': 0.85, 'FAR': 0.41379310344827586, 'FPR': 0.13186813186813187, 'TSS': 0.7181318681318681, 'HSS': 0.6108903935244441, 'Brier Score (BS)': 0.11449743136593453, 'Brier Skill Score (BSS)': 0.22487755392325282}
{'TP': 18, 'FP': 19, 'FN': 1, 'TN': 81, 'Accuracy': 0.8319327731092437, 'Precision': 0.4864864864864865, 'Recall': 0.9473684210526315, 'FAR': 0.5135135135135135, 'FPR': 0.19, 'TSS': 0.7573684210526315, 'HSS': 0.5473564082400981, 'Brier Score (BS)': 0.12787580112661245, 'Brier Skill Score (BSS)': 0.046921463287389775}
{'TP': 15, 'FP': 8, 'FN': 3, 'TN': 81, 'Accuracy': 0.897196261682243, 'Precision': 0.6521739130434783, 'Recall': 0.8333333333333334, 'FAR': 0.34782608695652173, 'FPR': 0.0898876404494382, 'TSS': 0.7434456928838952, 'HSS': 0.6692891242784795, 'Brier Score (BS)': 0.0946128081065603, 'Brier Skill Score (BSS)': 0.323831435697872}
{'TP': 18, 'FP': 9, 'FN': 4, 'TN'

In [9]:
for dataset_id in range(10):
    test_data, test_label = data_preprocess(f'/home/aistudio/Dataset/10_22_SHARP/{dataset_id}Test_all_class.csv')
    # test_data, test_label = SDO_ASO_test_data_preprocess(f'/home/aistudio/dataset/SDO_physics_features_Norm.csv')
    test_dataset = TensorDataset([test_data, test_label])
    test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)
    model = iTransformer(in_channels=in_channels,
                 num_features=num_features,
                 embed_dim=192,
                 n_heads=4, #4或8
                 depth=5,
                 mlp_ratio=4.0,
                 dropout=0.1,
                 num_class=2)
    calculator = MetricsCalculator()
    param = paddle.load(f'{save_dir}model/iTransformer_sift_BSS/iTransformer_{dataset_id}.pdparam')
    model.load_dict(param)
    model.eval()

    TSS = []
    # y_true = []
    # y_pred = []
    with paddle.no_grad():
        for img, label in test_loader:
            if mode == 0:
                pred = model(img[:, :, feature_start:feature_end])  # 修改
            else:
                pred = model(paddle.concat([img[:, :, feature_start:feature_end], img[:, :, 29:31]], axis=-1))
            # pred = model(paddle.concat([img[:, :, :, 1:2], img[:, :, :, 4:5],img[:, :, :, 22:27], img[:, :, :, 28:30]], axis=-1))
            label = label.astype('int32')
            pred_label = paddle.argmax(pred, axis=-1)
            pred_proba = F.softmax(pred, axis=-1)
            true_onehot = F.one_hot(label, 2)
            # pred_proba = F.softmax(pred, axis=-1)            
            calculator.update(label, pred_label, true_onehot, pred_proba)
    BSS_csv_path = f'{save_dir}BSS_csv/'
    os.makedirs(BSS_csv_path, exist_ok=True)
    pd.DataFrame(calculator.y_true_onehot).to_csv(f'{BSS_csv_path}y_true_{str(dataset_id)}_timestep_40.csv',index=False,header=False)
    pd.DataFrame(calculator.y_preds_proba).to_csv(f'{BSS_csv_path}y_pred_{str(dataset_id)}_timestep_40.csv',index=False,header=False)
    metric = calculator.calculate_metrics()
    print(metric)

    metric_folder = f'{save_dir}Metrics_timesteps_40/iTransformer_sift_BSS/'
    if not os.path.exists(metric_folder):
        os.makedirs(metric_folder)
    data_serializable = {k: int(v) if isinstance(v, np.integer) else v for k, v in metric.items()} 
    with open(metric_folder + f'dataset_{dataset_id}.json', 'w', encoding='utf-8') as f:  
        json.dump(data_serializable, f, ensure_ascii=True, indent=4)    

{'TP': 17, 'FP': 6, 'FN': 3, 'TN': 85, 'Accuracy': 0.918918918918919, 'Precision': 0.7391304347826086, 'Recall': 0.85, 'FAR': 0.2608695652173913, 'FPR': 0.06593406593406594, 'TSS': 0.7840659340659341, 'HSS': 0.7407215137795964, 'Brier Score (BS)': 0.07277765098309527, 'Brier Skill Score (BSS)': 0.507311297932573}
{'TP': 13, 'FP': 10, 'FN': 6, 'TN': 90, 'Accuracy': 0.865546218487395, 'Precision': 0.5652173913043478, 'Recall': 0.6842105263157895, 'FAR': 0.43478260869565216, 'FPR': 0.1, 'TSS': 0.5842105263157895, 'HSS': 0.5383123168324144, 'Brier Score (BS)': 0.08817351283000831, 'Brier Skill Score (BSS)': 0.3428288867443432}
{'TP': 16, 'FP': 8, 'FN': 2, 'TN': 81, 'Accuracy': 0.9065420560747663, 'Precision': 0.6666666666666666, 'Recall': 0.8888888888888888, 'FAR': 0.3333333333333333, 'FPR': 0.0898876404494382, 'TSS': 0.7990012484394506, 'HSS': 0.7052341578368204, 'Brier Score (BS)': 0.07686036110507581, 'Brier Skill Score (BSS)': 0.45070270019225145}
{'TP': 13, 'FP': 2, 'FN': 9, 'TN': 80,

In [10]:
std_file_name = f"{save_dir}timestep_40_iTransformer_TSS_BSS.txt"
sys.stdout = open(std_file_name, "w+", encoding="UTF-8")

read_folder = f'{save_dir}Metrics_timesteps_40/iTransformer_sift_TSS/'
read_file = sorted(os.listdir(read_folder))[:]
TSS = []
Accuracy = []
Recall = []
FAR = []
FPR = []
Percision = []
HSS = []
BS = []
BSS = []
for r_file in read_file:
    r_file = os.path.join(read_folder, r_file)
    with open(r_file, 'r') as f:
        data = json.load(f)
        print(data)
        TSS.append(data['TSS'])
        Accuracy.append(data['Accuracy'])
        Percision.append(data['Precision'])
        Recall.append(data['Recall'])
        FAR.append(data['FAR'])
        FPR.append(data['FPR'])
        HSS.append(data['HSS'])
        BS.append(data['Brier Score (BS)'])
        BSS.append(data['Brier Skill Score (BSS)'])

print(len(Recall))
print('Accuracy:', np.mean(Accuracy), np.std(Accuracy))
print('Percision:', np.mean(Percision), np.std(Percision))
print('Recall:', np.mean(Recall), np.std(Recall))
print('FAR:', np.mean(FAR), np.std(FAR))
print('FPR:', np.mean(FPR), np.std(FPR))
print('TSS:', np.mean(TSS), np.std(TSS))
print('HSS:', np.mean(HSS), np.std(HSS))
print('BS:', np.mean(BS), np.std(BS))
print('BSS:',np.mean(BSS),np.std(BSS))



In [11]:
print('------------------------------------------------------------------------')
sys.stdout.flush()
read_folder = f'{save_dir}Metrics_timesteps_40/iTransformer_sift_BSS/'
read_file = sorted(os.listdir(read_folder))[:]
TSS = []
Accuracy = []
Recall = []
FAR = []
FPR = []
Percision = []
HSS = []
BS = []
BSS = []
for r_file in read_file:
    r_file = os.path.join(read_folder, r_file)
    with open(r_file, 'r') as f:
        data = json.load(f)
        print(data)
        TSS.append(data['TSS'])
        Accuracy.append(data['Accuracy'])
        Percision.append(data['Precision'])
        Recall.append(data['Recall'])
        FAR.append(data['FAR'])
        FPR.append(data['FPR'])
        HSS.append(data['HSS'])
        BS.append(data['Brier Score (BS)'])
        BSS.append(data['Brier Skill Score (BSS)'])

print(len(Recall))
print('Accuracy:', np.mean(Accuracy), np.std(Accuracy))
print('Percision:', np.mean(Percision), np.std(Percision))
print('Recall:', np.mean(Recall), np.std(Recall))
print('FAR:', np.mean(FAR), np.std(FAR))
print('FPR:', np.mean(FPR), np.std(FPR))
print('TSS:', np.mean(TSS), np.std(TSS))
print('HSS:', np.mean(HSS), np.std(HSS))
print('BS:', np.mean(BS), np.std(BS))
print('BSS:',np.mean(BSS),np.std(BSS))
